# Hyperparameter tuning

In de reeds besproken machine learning technieken hebben we reeds een aantal keer vermeld dat er hyperparameters (denk aan regularisatieparameters, manieren van regularisatie, kernel type, ...).

In het geval van lineaire regressie gaat het dan over:
* L1 of L2 norm
* Regularisatieparameter $\lambda$
* learning rate

In het geval van SVM over:
* Type kernel
* Regularisatieparameter C
* Regularisatieparameter $\gamma$

Tot nu bestond de zoektocht naar de optimale combinatie van deze parameters door het manueel uitproberen en evalueren van een reeks combinaties van parameters.
Deze methode is echter niet schaalbaar en kan geautomatiseerd worden.
Dit gebeurd door middel van een gridsearch.

## Gridsearch

Het gridsearch algoritme bestaat eruit om een lijst op te stellen voor elke parameter welke waarden moeten getest worden.
Voor elke mogelijke combinatie van parameters gaat er dan een model getrained en geevalueerd worden.
Een voorbeeld van hoe dit kan geautomatiseerd worden binnen sklearn kan [hier](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html#sklearn.model_selection.GridSearchCV) gevonden worden.

In [3]:
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV
import numpy as np

iris = datasets.load_iris()

parameters = {
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'C': [0.1, 0.5, 0.9, 1, 1.5, 2, 10],
    'gamma': [0.1,0.5,1]
}

svm = svm.SVC()   # Classifier met Support-vector machines
gridsearch = GridSearchCV(svm, parameters, cv=3)

%time gridsearch.fit(iris.data, iris.target)
gridsearch.get_params()


CPU times: total: 500 ms
Wall time: 497 ms


{'cv': 3,
 'error_score': nan,
 'estimator__C': 1.0,
 'estimator__break_ties': False,
 'estimator__cache_size': 200,
 'estimator__class_weight': None,
 'estimator__coef0': 0.0,
 'estimator__decision_function_shape': 'ovr',
 'estimator__degree': 3,
 'estimator__gamma': 'scale',
 'estimator__kernel': 'rbf',
 'estimator__max_iter': -1,
 'estimator__probability': False,
 'estimator__random_state': None,
 'estimator__shrinking': True,
 'estimator__tol': 0.001,
 'estimator__verbose': False,
 'estimator': SVC(),
 'n_jobs': None,
 'param_grid': {'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
  'C': [0.1, 0.5, 0.9, 1, 1.5, 2, 10],
  'gamma': [0.1, 0.5, 1]},
 'pre_dispatch': '2*n_jobs',
 'refit': True,
 'return_train_score': False,
 'scoring': None,
 'verbose': 0}

In [4]:
gridsearch.best_params_

{'C': 1, 'gamma': 0.1, 'kernel': 'linear'}

De standaard methode van hierboven gaat alle combinaties afgaan.
Andere methoden die sneller maar niet alle combinaties aftoetsen zijn
* [RandomizedSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html#sklearn.model_selection.RandomizedSearchCV)
*[HalvingGridSearchCv](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.HalvingGridSearchCV.html#sklearn.model_selection.HalvingGridSearchCV)
*[HalvingRandomizedSearchCv](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.HalvingRandomSearchCV.html#sklearn.model_selection.HalvingRandomSearchCV)

Belangrijk om hierbij op te merken is dat het GridSearch algoritme enkel verschillende parameters van het model trained en dat er geen eigenschappen van de data kan veranderd worden.
Indien je ook een exhausieve search wilt doen van het aantal hogere orde features of de vorm van scaling die gebruikt wordt op input parameters. Moet je een eigen wrapper schrijven die nog deze zaken uittest en de performantie van de uiteindelijke modellen vergelijkt.

## Validatieset

Welke data kunnen we nu gebruiken om deze gridsearch te evalueren.
Zowel de testdata als de trainingsdata kan niet gebruikt worden omdat we niet kunnen evalueren op de data waarmee het model getrained is.
Om deze reden wordt de dataset typisch in drie opgedeeld, namelijk een training-, test- en validatieset.
De validatieset is de data die dan gebruikt kan worden voor hyperparameter tuning.
Typisch wordt de dataset dan in de volgende groottes opgedeeld:
* Testset: 15%
* Validatieset: 15% 
* Trainingsdata: 70%

Dit zijn echter geen vaste waarden en kunnen wat verschillen in de praktijk.
Hoe meer data je beschibaar is hoe groter het percentage trainingsdata kan zijn. 
In het geval van big-data applicaties kan dit oplopen tot 98%.

## K-fold cross validation

Bij het steeds gebruiken van dezelfde validatieset is het mogelijk dat er een unieke split is die leidt tot een onverwacht goed of slecht resultaat.
Om dit tegen te gaan kan er gebruik gemaakt worden van K-fold cross validation.
Daarbij berekenen we de verwachte error K keer, elke keer met een andere train en validatie set om zo de kans te verhogen dat het uiteindelijke model ook goed werkt op de testset met ongeziene data.
Standaard wordt er bij het gebruik van het gridsearch algoritme gebruik gemaakt van 5 folds voor het zoeken naar de beste hyperparameters.
Indien de standaard manier niet voldoet voor de gewenste toepassing kan je ook de split rechtstreeks uitvoeren.
Meer informatie over deze methode vind je [hier](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html)

![kfold cross validation](images/kFold.png)

## Oefening

In [deze dataset](https://www.kaggle.com/mathchi/diabetes-data-set) is een hele reeks data beschikbaar over een aantal medische eigenschappen van personen en of deze personen diabetes hebben of niet.
Ga nu op zoek naar het beste model om te voorspellen of een persoon diabetes gaat hebben of niet.
Test hierbij zowel de logistische regressie en svm methoden en maak gebruik van gridsearch met 10-fold cross validation om de verschillende hyperparameters te testen. 

Wat is de hoogst behaalde accuraatheid en de benodigde hyperparameters?

Indien dit gelukt is, zoek ook het model dat de hoogste weighted f1-score behaald. 
Welke techniek gebruikte dit model en welke hyperparameters zijn er hiervoor gekozen?
Vergelijk beide modellen. Is er een significant verschil in de resulterende hyperparameters?
Is de behaalde accuraatheid sterk afwijkend?

In [16]:
import opendatasets as od
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

In [ ]:
#od.download("https://www.kaggle.com/mathchi/diabetes-data-set")

In [ ]:
df = pd.read_csv("./diabetes-data-set/diabetes.csv")
display(df.head())
y = df.Outcome
X = df.drop("Outcome", axis=1).values

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)

## Pipeline

Bij het werken met een gridsearch is er nog een probleem omtrent data leakage.
Omdat we in bovenstaande voorbeeld de scaling hebben gedaan op de volledige dataset is alle data in principe gebruikt om de scaling uit te voeren en dus gaat er steeds een effect zijn van de training-folds op de validation-fold wat niet gewenst is.
Dit kan onterecht de score verbeteren.

Om dit tegen te gaan kan men gebruik maken van Pipelines. Hiermee definieer je de flow die de data moet ondergaan om verwerkt te worden. De volledige pipeline of flow kan dan gegeven worden aan de gridsearch algoritme. Hierdoor worden de preprocessing stappen (zoals scalers, ordinal of one-hotencoder) steeds enkel op de training folds uitgevoerd.
Dit zorgt ervoor dat er geen data leakage kan optreden. 
Het bijkomende voordeel is dat ook parameters van de preprocessing stappen meegenomen kunnen worden in de gridsearch.

Let op dat NaN waarden invullen kan gebeuren in de pipeline maar rijen of kolommen weglaten kan niet gedaan worden tijdens de pipeline. Echter is het gebruik van een pipeline een goede stap in het verhogen van de leesbaarheid van een code stuk en wordt het daardoor veelvuldig gebruikt.

In [27]:
import numpy as np

from sklearn.compose import ColumnTransformer
from sklearn.datasets import fetch_openml
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV

np.random.seed(0)

X, y = fetch_openml("titanic", version=1, as_frame=True, return_X_y=True)
X

,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1.0,"Allen, Miss. Elisabeth Walton",female,29.0000,0.0,0.0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1.0,"Allison, Master. Hudson Trevor",male,0.9167,1.0,2.0,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1.0,"Allison, Miss. Helen Loraine",female,2.0000,1.0,2.0,113781,151.5500,C22 C26,S,None,NaN,"Montreal, PQ / Chesterville, ON"
3,1.0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1.0,2.0,113781,151.5500,C22 C26,S,None,135.0,"Montreal, PQ / Chesterville, ON"
4,1.0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1.0,2.0,113781,151.5500,C22 C26,S,None,NaN,"Montreal, PQ / Chesterville, ON"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304,3.0,"Zabour, Miss. Hileni",female,14.5000,1.0,0.0,2665,14.4542,None,C,None,328.0,None
1305,3.0,"Zabour, Miss. Thamine",female,NaN,1.0,0.0,2665,14.4542,None,C,None,NaN,None
1306,3.0,"Zakarian, Mr. Mapriededer",male,26.5000,0.0,0.0,2656,7.2250,None,C,None,304.0,None
1307,3.0,"Zakarian, Mr. Ortin",male,27.0000,0.0,0.0,2670,7.2250,None,C,None,NaN,None


In [14]:
X.body.unique()

array([ nan, 135.,  22., 124., 148., 208., 172., 269.,  62., 133., 275.,
       147., 110., 307.,  38.,  80.,  45., 258., 126., 292., 175., 249.,
       230., 122., 263., 234., 189., 166., 207., 232.,  16., 109.,  96.,
        46., 245., 169., 174.,  97.,  18., 130.,  17., 295., 286., 236.,
       322., 297., 155., 305.,  19.,  75.,  35., 256., 149., 283., 165.,
       108., 121.,  52., 209., 271.,  43.,  15., 101., 287.,  81., 294.,
       293., 190.,  72., 103.,  79., 259., 260., 142., 299., 171.,   9.,
       197.,  51., 187.,  68.,  47.,  98., 188.,  69., 306., 120., 143.,
       156., 285.,  37.,  58.,  70., 196., 153.,  61.,  53., 201., 309.,
       181., 173.,  89.,   4., 206., 327., 119.,   7.,  32.,  67., 284.,
       261., 176.,  50.,   1., 255., 298., 314.,  14., 131., 312., 328.,
       304.])

In [37]:
# verdeel de aanwezige kolommen in groepjes op basis van de data in de kolommen
numeric_features = ['age', 'fare', 'body'] # body is een vorm van data leakage (lichaam gevonden en welk het is dus data die pas later gekend is)
minmax_features = ['pclass', 'sibsp', 'parch'] # pclass is ordinal encoded, we willen dit enkel geschaald worden met behoud van de afstand
categorical_features = ['sex', 'embarked'] # text naar getal
dropped_features = ['name', 'ticket', 'cabin', 'boat', 'home.dest']

# stel de pipelines op om elke soort kolom te verwerken
pipeline_numeric = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')), # onbekende waarden invullen met waarden, want ML-technieken kunnen niet altijd goed omgaan met Null/Nan
    ('scaler', StandardScaler())
])

pipeline_minmax= Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value=-1)), # invullen met een niet voorkomende categorie
    ('scaler', MinMaxScaler())
])


pipeline_categorical= Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='unknown')), # invullen met een niet voorkomende categorie
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# groepeer de preprocessors per soort kolom in een columntransformer
# kolommen die niet in de columntransformer staan worden standaard niet doorgegeven (kan aangepast worden met de remainder parameter)
preprocessor = ColumnTransformer(
    transformers = [
        ('numeric', pipeline_numeric, numeric_features),   # doe de pipeline_numeric voor de numeric_features
        ('minmax', pipeline_minmax, minmax_features),
        ('categorical', pipeline_categorical, categorical_features),
    ]
)

clf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(solver='saga', max_iter=1000))
])

# best eerst X en y splitsen in train en test
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)

clf.fit(X_train, y_train)
print('model score:', clf.score(X_test, y_test))



model score: 0.7480916030534351


In [38]:
param_grid = {
    'classifier__penalty': ['l1', 'l2', 'elasticnet'],     # twee keer underscore tussen niveaus in de pipeline
    'classifier__C': [0.1,0.6, 1, 1.4, 3],
    'preprocessor__numeric__imputer__strategy': ['mean', 'median'] # pas de strategy parameter aan in de imputer van de numeric pipeline in de preprocessor
}

# param_grid kan ook een lijstje zijn van dictionaries.
# elke set is een dictionary met opties die uitgevoerd worden zoals hierboven
#param_grid = [
#    set_1, set_2, set_3
#]
# dit is een mogelijkheid om opties te nemen die niet gecombineerd worden

gridsearch = GridSearchCV(clf, param_grid, cv=4, scoring='accuracy')
gridsearch.fit(X_train, y_train)

print('model score:', gridsearch.score(X_test, y_test))

model score: 0.7557251908396947


C:\Users\jens.baetens3\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
40 fits failed out of a total of 120.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
40 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\jens.baetens3\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\jens.baetens3\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\pipeline.py", line 382, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "C:\Users\jens.baetens3\AppData\Local

## Oefening

Maak een pipeline voor een gridsearch uit te voeren op de diabetes-dataset in sklearn. Gebruik een Min-Max scaler voor de numerieke waarden en een ordinal encoder voor de categorieke kolommen.
Zoek naar de beste combinatie van hyperparameters van een SVM-classifier door middel van een grid search met cross validation.